# 🎯 Modelo Dummy Completo - Demo DSDL

**Objetivo**: Demo completo del flujo Dev → Prod con modelo funcional

**Modelo**: Autoencoder para detección de anomalías

**Nota**: Este es un modelo de DEMO para validar el flujo completo


In [ ]:
# Imports necesarios
import sys
sys.path.append("/srv/notebooks_custom/helpers")

# Helpers custom
from telemetry_helper import log_metrics, log_training_step, log_error, log_prediction
from metrics_calculator import calculate_all_metrics

# Librerías estándar
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("✅ Todos los imports exitosos")


In [ ]:
# ═══════════════════════════════════════════════════════════════
# FUNCIÓN 1: INIT - Inicialización del modelo
# ═══════════════════════════════════════════════════════════════

def init(param):
    """
    Inicializar modelo y parámetros globales
    
    Args:
        param: Diccionario con parámetros del modelo
    
    Returns:
        model: Modelo inicializado
    """
    global model, scaler, n_features
    
    try:
        print(f"🔧 Inicializando modelo con parámetros: {param}")
        n_features = None
        model = None
        scaler = None
        print("✅ Modelo inicializado (autoencoder para detección de anomalías)")
        return model
        
    except Exception as e:
        log_error(model_name='demo_modelo_completo', error_message=str(e), error_type='init')
        raise e


In [ ]:
# ═══════════════════════════════════════════════════════════════
# FUNCIÓN 2: FIT - Entrenamiento del modelo
# ═══════════════════════════════════════════════════════════════

def fit(df, param):
    """
    Entrenar modelo con datos
    
    Args:
        df: DataFrame con datos de entrenamiento
        param: Diccionario con parámetros de entrenamiento
    
    Returns:
        model: Modelo entrenado
    """
    global model, scaler, n_features
    
    try:
        print(f"📊 Datos recibidos: {df.shape}")
        
        # Detectar features
        feature_cols = df.columns.tolist()
        n_features = len(feature_cols)
        X = df[feature_cols].values
        
        # Preprocesamiento
        print("🔧 Preprocesando datos...")
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Split train/val
        X_train, X_val = train_test_split(X_scaled, test_size=0.2, random_state=42)
        
        # Construir modelo (autoencoder)
        print(f"🔧 Construyendo autoencoder con {n_features} features...")
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_shape=(n_features,)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(n_features, activation='sigmoid')
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=param.get('learning_rate', 0.001)),
            loss='mse',
            metrics=['mae', 'mse']
        )
        
        # Entrenar
        epochs = param.get('epochs', 50)
        batch_size = param.get('batch_size', 32)
        
        history = model.fit(
            X_train, X_train,
            validation_data=(X_val, X_val),
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
        )
        
        # Calcular métricas
        val_pred = model.predict(X_val, verbose=0)
        mse = np.mean((X_val - val_pred) ** 2)
        mae = np.mean(np.abs(X_val - val_pred))
        
        print(f"📊 MSE: {mse:.4f}, MAE: {mae:.4f}")
        
        # Telemetría
        log_metrics(model_name='demo_modelo_completo', mae=mae, mse=mse)
        log_training_step(model_name='demo_modelo_completo', epoch=epochs, loss=mse)
        
        print("✅ Modelo entrenado exitosamente")
        return model
        
    except Exception as e:
        log_error(model_name='demo_modelo_completo', error_message=str(e), error_type='fit')
        raise e


In [ ]:
# ═══════════════════════════════════════════════════════════════
# FUNCIÓN 3: APPLY - Aplicar modelo a nuevos datos
# ═══════════════════════════════════════════════════════════════

def apply(df):
    """
    Aplicar modelo entrenado para predicción
    
    Args:
        df: DataFrame con datos para predecir
    
    Returns:
        errors: Array con errores de reconstrucción
    """
    global model, scaler
    
    try:
        if model is None or scaler is None:
            raise ValueError("Modelo no entrenado. Ejecuta fit() primero")
        
        feature_cols = df.columns.tolist()
        X = df[feature_cols].values
        X_scaled = scaler.transform(X)
        
        reconstructed = model.predict(X_scaled, verbose=0)
        errors = np.mean((X_scaled - reconstructed) ** 2, axis=1)
        
        log_prediction(model_name='demo_modelo_completo', num_predictions=len(errors))
        
        return errors
        
    except Exception as e:
        log_error(model_name='demo_modelo_completo', error_message=str(e), error_type='apply')
        raise e


In [ ]:
# ═══════════════════════════════════════════════════════════════
# FUNCIÓN 4: SUMMARY - Resumen del modelo
# ═══════════════════════════════════════════════════════════════

def summary(df):
    """
    Generar resumen de métricas del modelo
    
    Args:
        df: DataFrame (opcional)
    
    Returns:
        summary_dict: Diccionario con resumen del modelo
    """
    global model, scaler
    
    try:
        if model is None:
            return {"status": "Model not initialized"}
        
        return {
            "model_type": "Autoencoder",
            "use_case": "Anomaly Detection",
            "trainable_parameters": model.count_params(),
            "layers": len(model.layers),
            "features": n_features if 'n_features' in globals() else None
        }
        
    except Exception as e:
        log_error(model_name='demo_modelo_completo', error_message=str(e), error_type='summary')
        return {"error": str(e)}


## 🔄 Flujo Completo: Splunk → Modelo → Producción

**Este notebook tiene 2 modos de uso:**

1. **Modo Testing Local** (abajo): Con datos sintéticos
2. **Modo Producción con Splunk**: Las funciones `fit()` y `apply()` reciben datos automáticamente de Splunk

### ⚠️ IMPORTANTE: Las funciones `init`, `fit`, `apply`, `summary` NO consultan Splunk manualmente
### DSDL automáticamente les pasa los datos cuando ejecutas:
### `| fit MLTKContainer algo=demo_modelo_completo ... into app:modelo_v1`

---

## 🧪 Opcional: Consultar Splunk Interactivamente (Para Exploración)

Si quieres consultar datos de Splunk desde el notebook (modo interactivo de exploración):


In [ ]:
# OPCIONAL: Consultar Splunk interactivamente (solo para exploración/debug)
# ⚠️ Esto NO es necesario para producción, DSDL ya lo hace automáticamente

try:
    from dsdlsupport import SplunkSearch
    
    # Consultar datos de Splunk
    search = SplunkSearch.SplunkSearch(
        search='index=demo_anomalias_data | head 100 | table feature_*'
    )
    
    # Obtener como DataFrame
    df_sample = search.as_df()
    
    print(f"✅ Muestra de Splunk: {df_sample.shape}")
    print(f"   Columnas: {df_sample.columns.tolist()}")
    print("\nPrimeras 5 filas:")
    print(df_sample.head())
    
except Exception as e:
    print(f"⚠️  SplunkSearch no disponible o no configurado: {e}")
    print("   (Esto es normal si no tienes Splunk Access configurado)")


In [ ]:
# Generar datos dummy para testing
np.random.seed(42)

# Datos normales
normal_data = np.random.normal(0, 1, (1000, 5))
df_normal = pd.DataFrame(normal_data, columns=[f'feature_{i}' for i in range(5)])

# Anomalías
anomaly_data = np.random.normal(5, 2, (100, 5))
df_anomaly = pd.DataFrame(anomaly_data, columns=[f'feature_{i}' for i in range(5)])

df_test = pd.concat([df_normal, df_anomaly], ignore_index=True)

print(f"✅ Datos: {df_test.shape} (1000 normales, 100 anómalos)")


In [ ]:
# Test completo: INIT → FIT → APPLY → SUMMARY
print("=" * 60)
print("🧪 TEST COMPLETO DEL MODELO")
print("=" * 60)

param = {'epochs': 20, 'batch_size': 32, 'learning_rate': 0.001}

# 1. INIT
print("\n1️⃣  INIT")
model = init(param)

# 2. FIT
print("\n2️⃣  FIT")
model = fit(df_normal, param)

# 3. APPLY
print("\n3️⃣  APPLY")
predictions = apply(df_test)
print(f"✅ Predicciones: {len(predictions)} valores")

# 4. SUMMARY
print("\n4️⃣  SUMMARY")
summary_result = summary(df_test)
print(f"✅ Summary: {summary_result}")

print("\n" + "=" * 60)
print("✅ TEST COMPLETO EXITOSO")
print("=" * 60)
